In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import time
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt

# 1. Dataset Preprocessing
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)


100%|██████████| 170498071/170498071 [00:01<00:00, 96295425.58it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
class ViT(nn.Module):
    def __init__(self, img_size=32, patch_size=4, num_classes=10, dim=64, depth=6, heads=8, mlp_dim=128):
        super(ViT, self).__init__()
        assert img_size % patch_size == 0, "Image dimensions must be divisible by the patch size."
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_dim = 3 * patch_size ** 2

        self.patch_embeddings = nn.Linear(self.patch_dim, dim)
        self.position_embeddings = nn.Parameter(torch.randn(1, self.num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        
        # Define the encoder layers
        encoder_layer = nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=depth)
        
        self.to_cls_token = nn.Identity()
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, x):
        b, c, h, w = x.shape
        patch_size = int((h * w) // self.num_patches)
        x = x.unfold(2, patch_size, patch_size).unfold(3, patch_size, patch_size)
        x = x.contiguous().view(b, -1, self.patch_dim)  # Shape: [batch_size, num_patches, patch_dim]
        
        x = self.patch_embeddings(x)  # Apply patch embedding to get [batch_size, num_patches, dim]

        cls_tokens = self.cls_token.expand(b, -1, -1)  # Shape: [batch_size, 1, dim]
        x = torch.cat((cls_tokens, x), dim=1)  # Concatenate class token
        x += self.position_embeddings

        x = self.transformer_encoder(x)  # Pass through the encoder
        x = self.to_cls_token(x[:, 0])
        return self.mlp_head(x)


In [7]:
# 3. Model Training and Checkpointing
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    start_time = time.time()
    
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    training_time = time.time() - start_time
    avg_loss = running_loss / len(train_loader)
    
    return avg_loss, training_time

# 4. Evaluation and Metrics Logging
def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)

    return accuracy, precision, recall, f1, cm

# 5. Training Loop with Checkpointing and Logging
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ViT().to(device)
model = nn.DataParallel(model)  # Enable multi-GPU support
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20
best_f1 = 0.0
train_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    train_loss, training_time = train_model(model, train_loader, criterion, optimizer, device)
    accuracy, precision, recall, f1, cm = evaluate_model(model, test_loader, device)
    
    train_losses.append(train_loss)
    val_accuracies.append(accuracy)
    
    # Save model at each epoch, overwriting the previous checkpoint
    torch.save(model, './model_checkpoint.pth')  # Save the entire model
    
    # Update best F1 score for reference (if needed)
    if f1 > best_f1:
        best_f1 = f1
    
    # Log metrics to file
    with open('metrics_log.txt', 'a') as f:
        f.write(f'Epoch: {epoch+1}, Loss: {train_loss}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1}, Time: {training_time}\n')

    print(f'Epoch {epoch+1}/{num_epochs} - Loss: {train_loss:.4f} - Accuracy: {accuracy:.4f} - F1-score: {f1:.4f}')
    print(f'Confusion Matrix:\n{cm}')


# 6. Plotting Training and Validation Curves
plt.figure(figsize=(12, 5))

# Plot Loss Curve
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.legend()

# Plot Accuracy Curve
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy Curve')
plt.legend()

plt.tight_layout()
plt.savefig('training_validation_curves.png')
plt.show()


100%|██████████| 79/79 [00:02<00:00, 32.42it/s]


Epoch 1/10 - Loss: 2.1641 - Accuracy: 0.2753 - F1-score: 0.2542
Confusion Matrix:
[[300  41  41  20  22 125  40  14 339  58]
 [ 42 255  13  55  30 150  77  40 173 165]
 [ 72  36 114  29 127 231 260  23  74  34]
 [ 38  68  39  68  75 355 230  39  38  50]
 [ 25  32  70  30 172 185 364  34  60  28]
 [ 34  46  49  54  52 477 178  31  43  36]
 [ 17  31  52  31 121 207 452  27  26  36]
 [ 27  64  32  43  73 265 183  63  79 171]
 [139  69  23  11  24 111  16   9 497 101]
 [ 41 116  13  36  22 107  66  38 206 355]]


  0%|          | 0/391 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
100%|██████████| 79/79 [00:02<00:00, 32.25it/s]


Epoch 2/10 - Loss: 1.9332 - Accuracy: 0.3269 - F1-score: 0.3108
Confusion Matrix:
[[411  43  66   7  27  58  19  13 300  56]
 [ 61 298  22  32  27  71  50  41 214 184]
 [109  45 226  31 149 159 181  18  49  33]
 [ 52  57 130  72  83 328 136  60  28  54]
 [ 45  26 190  29 312  78 201  40  52  27]
 [ 45  30 102  48  76 463 130  47  36  23]
 [ 16  26 139  29 139 137 414  52  16  32]
 [ 42  69 101  36 113 192 100 128  57 162]
 [158  72  29  10  23  43   8   7 569  81]
 [ 59 122  22  18  32  61  47  56 207 376]]


  0%|          | 0/391 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
100%|██████████| 79/79 [00:02<00:00, 32.19it/s]


Epoch 3/10 - Loss: 1.8479 - Accuracy: 0.3442 - F1-score: 0.3265
Confusion Matrix:
[[260  51  29  49  34  90  27  25 324 111]
 [ 13 411   4  28  11  83  40  67 113 230]
 [ 57  38 111  42 175 224 204  66  43  40]
 [ 13  61  45  78  62 403 161  98  21  58]
 [ 19  32  78  23 293 135 257  79  43  41]
 [  4  36  39  39  39 565 131  94  24  29]
 [  4  38  45  40  96 170 471  96   6  34]
 [  9  71  18  41  85 216 113 274  22 151]
 [ 65 121   4  22  19  64  13  24 538 130]
 [ 12 178   6  23  11  79  37  97 116 441]]


  0%|          | 0/391 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
  0%|          | 0/79 [00:00<?, ?it/s]Exception in thread Thread-3620 (_pin_memory_loop):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner

    self.run()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 55, in _pin_memory_loop
    do_one_step()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data

KeyboardInterrupt: 

# Fine-tuning for additional epochs using LR scheduler

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Define device (ensure it uses only one GPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the entire model as it was saved
model_path = '/kaggle/working/model_checkpoint.pth'
model = torch.load(model_path, map_location=device)  # Load full model directly
model = model.to(device)

# Define optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)

# Training loop as before
num_epochs = 20
best_f1 = 0.0
train_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    # Training phase
    train_loss, training_time = train_model(model, train_loader, criterion, optimizer, device)
    accuracy, precision, recall, f1, cm = evaluate_model(model, test_loader, device)
    
    train_losses.append(train_loss)
    val_accuracies.append(accuracy)
    
    # Save the full model checkpoint for retraining or deployment
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model, '/kaggle/working/data/full_model_checkpoint_2.pth')  # Save the full model
    
    # Step the scheduler based on validation accuracy
    scheduler.step(accuracy)
    
    # Log metrics to file
    with open('metrics_log_continued.txt', 'a') as f:
        f.write(f'Epoch: {epoch+1}, Loss: {train_loss}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1}, Time: {training_time}\n')

    print(f'Epoch {epoch+1}/{num_epochs} - Loss: {train_loss:.4f} - Accuracy: {accuracy:.4f} - F1-score: {f1:.4f}')
    print(f'Confusion Matrix:\n{cm}')


/tmp/ipykernel_30/1750144328.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)  # Load full model directly
  7%|▋     

Epoch 1/20 - Loss: 1.7921 - Accuracy: 0.3635 - F1-score: 0.3487
Confusion Matrix:
[[421  76  20  16  45  26  18  34 296  48]
 [ 41 528   1  22  15  12  20  57 155 149]
 [113  61 117  52 274  88 130  91  34  40]
 [ 55 106  35 129 106 211 108 155  32  63]
 [ 51  50  49  30 440  54 135 117  49  25]
 [ 41  60  37  92 109 346  90 141  43  41]
 [ 19  58  30  46 222  79 361 132  14  39]
 [ 40 113  22  48 128  84  53 338  50 124]
 [146 140   3  15  27  14   2  16 579  58]
 [ 43 260   4  20  14  21  17  79 166 376]]


  0%|          | 0/391 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
100%|██████████| 79/79 [00:02<00:00, 27.75it/s]


Epoch 2/20 - Loss: 1.7587 - Accuracy: 0.3731 - F1-score: 0.3615
Confusion Matrix:
[[441  53  26  22  53  19  19  35 281  51]
 [ 43 442   3  27  20  24  32  47 204 158]
 [104  32 134  80 281  91 124  80  42  32]
 [ 55  55  38 162 118 231 127 113  44  57]
 [ 51  30  65  35 452  40 148 102  56  21]
 [ 31  27  50 144  98 367  91 115  50  27]
 [ 14  31  38  66 217  86 385 106  23  34]
 [ 36  72  38  69 133 110  58 357  42  85]
 [131  90   7  14  25  20   6  17 626  64]
 [ 48 212   5  27  22  21  30  84 186 365]]


  0%|          | 0/391 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
100%|██████████| 79/79 [00:02<00:00, 31.93it/s]


...

Epoch 18/20 - Loss: 1.5148 - Accuracy: 0.4626 - F1-score: 0.4576
Confusion Matrix:
[[485  39  79  10  41  22  30  36 211  47]
 [ 48 509  14  24  23  12  22  57  95 196]
 [ 65  16 365  57 146 103 127  73  29  19]
 [ 26  12 105 195  72 281 172  82  19  36]
 [ 37  11 195  29 380  52 146 104  34  12]
 [ 15  10 104 103  70 434 119 109  24  12]
 [  9   9 100  53 135  55 562  52   8  17]
 [ 23  19  47  44 115 102  52 543  11  44]
 [ 85  72  18  15  30  12  11  26 665  66]
 [ 37 160  11  32  21  19  43  81 108 488]]


  0%|          | 0/391 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
100%|██████████| 79/79 [00:02<00:00, 33.15it/s]


Epoch 19/20 - Loss: 1.5062 - Accuracy: 0.4695 - F1-score: 0.4645
Confusion Matrix:
[[513  51  83  21  23  26  35  25 164  59]
 [ 54 584  13  23   8  11  29  19  66 193]
 [ 79  19 371  70 104 106 153  56  18  24]
 [ 27  26 109 254  31 284 159  61  12  37]
 [ 41  11 229  42 275  70 188 101  30  13]
 [ 11   9  94 157  34 479 111  72  15  18]
 [  3  13 106  59  65  56 656  16   9  17]
 [ 27  33  55  59  81 136  69 467   7  66]
 [112  90  21  17  19  28  14   9 607  83]
 [ 47 205  14  36   8  24  48  46  83 489]]


  0%|          | 0/391 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
100%|██████████| 79/79 [00:02<00:00, 30.59it/s]

Epoch 20/20 - Loss: 1.5004 - Accuracy: 0.4615 - F1-score: 0.4517
Confusion Matrix:
[[521  73  49  16  21  13  40   7 212  48]
 [ 20 652   8  25   4   3  42  12  82 152]
 [104  35 248  99 105  63 227  58  30  31]
 [ 37  37  50 300  28 173 243  41  32  59]
 [ 49  23 126  60 298  28 284  65  37  30]
 [ 27  13  66 218  34 348 172  62  27  33]
 [ 10  19  52  71  50  21 711  23  16  27]
 [ 37  48  24  93  88  79 106 415  26  84]
 [ 90  95   5  17  14   7  19   5 672  76]
 [ 37 278   5  36   6  10  52  25 101 450]]
